For each SNR level, let's analyse the precision & recall (mono vs beamformed)...

In [4]:
# Some useful functions / imports

import json
from pprint import pprint

def read_results_from_file(file_path):
    """Read the BirdNET detections from a file"""

    with open(file_path, 'r') as file:
        results_dict = json.load(file)

    return results_dict

In [11]:
# First, for a single SNR level------------------------------------------------

expected_species_count = 30         # We should be able to detect 30 calls from each species, per file (3 calls per speaker, over 10 of the 31 speakers in the sphere)

folder_path = "data/processed/speaker_sphere_lab_tests/0SNR"

counts_file_path = folder_path + "/species_counts.json"

counts_dic = read_results_from_file(counts_file_path)

# pprint(counts_dic)

def get_TP_FP_FN(species_count_dict):
    """Calculates the True Positives, True Negatives & False Negatives, from a single processed.json file
    --> Assumes that all Carolina Wren and Eurasian Blackhat detections are correct (as we're expecting them) = TP
    --> Assumes all non-CW and non-EB detections = FP (as weren't expecting other species present)   
    --> Assumes that FN = expected count - TP (i.e., all the missed detections)
    Returns: TP, FP & FN for both channels, in a dictionary"""

    # Initialise dictionary
    P_N_dict = {"mono_channel": {"TP": 0, "FP": 0, "FN": 0},
                "beamformed": {"TP": 0, "FP": 0, "FN": 0}}
    
    for channel in species_count_dict.keys():
        chan_data = species_count_dict[channel]
        # Initialise counts
        TP = 0
        FP = 0
        FN = 0
        for species in chan_data.keys():
            species_count = chan_data[species]
            if species == "Eurasian Blackcap" or species == "Carolina Wren":
                TP += species_count
            else:
                FP += species_count
        FN = expected_species_count*2 - TP          # 2 species expected, thus *2
        
        P_N_dict[channel]["TP"] = TP
        P_N_dict[channel]["FP"] = FP
        P_N_dict[channel]["FN"] = FN
    
    return P_N_dict


def calculate_precision(TP, FP):
    """Returns the precision score, to 3 dp"""

    return round(TP/(TP+FP), 3)


def calculate_recall(TP, FN):
    """Returns the recall rate, to 3 dp"""

    return round(TP/(TP+FN), 3)


def calculate_f1(precision, recall):
    """Returns an f1 score, to 3 dp"""

    return round((2*precision*recall)/(precision+recall), 3)
    

def get_precision_recall_f1(P_N_dict):
    """Calculates the precision, recall rate, and f1 score, for each channel
    Returns: Dictionary with scores, for each channel (and maintains TP, FP & FN)"""
    
    for channel in P_N_data.keys():
        channel_data = P_N_data[channel]
        prec = calculate_precision(channel_data["TP"], channel_data["FP"])
        rec = calculate_recall(channel_data["TP"], channel_data["FN"])
        f1 = calculate_f1(prec, rec)
        
        P_N_dict[channel]["precision"] = prec
        P_N_dict[channel]["recall"] = rec
        P_N_dict[channel]["f1"] = f1

    return P_N_dict

P_N_data = get_TP_FP_FN(counts_dic)
# pprint(P_N_data)



prec_rec_data = get_precision_recall_f1(P_N_data)
pprint(P_N_data)


{'beamformed': {'FN': 33,
                'FP': 1,
                'TP': 27,
                'f1': 0.614,
                'precision': 0.964,
                'recall': 0.45},
 'mono_channel': {'FN': 42,
                  'FP': 0,
                  'TP': 18,
                  'f1': 0.462,
                  'precision': 1.0,
                  'recall': 0.3}}


In [15]:
snr_list = ["0", "5", "10", "15", "20", "n5", "n10", "n15", "n20"]

folder_path = "data/processed/speaker_sphere_lab_tests/"

def extract_prec_rec(SNR_level):
    """For a single species_count.json file, calculate the precision, recall & f1"""

    file_path = folder_path + SNR_level + "SNR/species_counts.json"

    counts_dic = read_results_from_file(file_path)

    P_N_data = get_TP_FP_FN(counts_dic)
    prec_rec_data = get_precision_recall_f1(P_N_data)

    return prec_rec_data

SNR_prec_rec_data = {}

for snr in snr_list:
    SNR_prec_rec_data[snr] = extract_prec_rec(snr)

pprint(SNR_prec_rec_data)
    

{'0': {'beamformed': {'FN': 33,
                      'FP': 1,
                      'TP': 27,
                      'f1': 0.614,
                      'precision': 0.964,
                      'recall': 0.45},
       'mono_channel': {'FN': 42,
                        'FP': 0,
                        'TP': 18,
                        'f1': 0.462,
                        'precision': 1.0,
                        'recall': 0.3}},
 '10': {'beamformed': {'FN': 9,
                       'FP': 1,
                       'TP': 51,
                       'f1': 0.614,
                       'precision': 0.964,
                       'recall': 0.45},
        'mono_channel': {'FN': 26,
                         'FP': 0,
                         'TP': 34,
                         'f1': 0.462,
                         'precision': 1.0,
                         'recall': 0.3}},
 '15': {'beamformed': {'FN': 5,
                       'FP': 0,
                       'TP': 55,
                       'f1':